# Data Engineering 1: Lab 06
---------------

### Task 01: Consistent Hashing
#### Fill the parts marked with Todo for a complete implementation of Consistent Hashing

In [152]:
import names
import pandas as pd


#returns list of size n with random names
def getData(n):
    data = []
    i = 0
    while i < n:
        data.append(names.get_first_name())
        i += 1
    return data

#returns list of hash values for list of data
def getHashs(data):
    hashs = []
    for word in data:
        hashs.append(abs(hash(word)) % (10 ** 8))
    return hashs

#returns list of size m with random server names
def getServers(m):
    data = []
    i = 0
    while i < m:
        data.append('Server_' + names.get_first_name())
        i += 1
    return data

def getAngles(data):
    angle = []
    for hash_data in data:
        angle.append((360 / (10 ** 8)) * hash_data)
    return angle


#TodO: create a DataFrame with the list created with the function getdata(100) 
# and a type column filled with the type key

df_names = pd.DataFrame(getData(100))
df_names.rename(columns={0:'name'}, inplace=True)
df_names['type'] = 'key'
    
#Todo: add a column to the DataFrame with the hashs created with the getHashs() method

hashs = getHashs(df_names['name'])
df_names['hash'] = hashs

#Todo: create m servers with the getServers function and add them to the DataFrame with the type server
#Todo: add a column to the DataFrame with the angle on the circle

df_servers = pd.DataFrame(getServers(10))
df_servers.rename(columns={0:'name'}, inplace=True)
df_servers['type'] = 'server'
hashs = getHashs(df_servers['name'])
df_servers['hash'] = hashs

df_servers['angle'] = getAngles(df_servers['hash'])
df_names['angle'] = getAngles(df_names['hash'])

#Todo: add a column to the DataFrame with the corresponding server

df_names = df_names.sort_values('hash')
df_servers = df_servers.sort_values('hash')

def AssignNamesToServers():
    for index_server, row_server in df_servers.iterrows():
        for index_names, row_names in df_names.iterrows():
            if row_names['type'] == 'key':
                if df_names.at[index_names, 'angle'] > df_servers.at[index_server, 'angle']:
                    df_names.at[index_names, 'assigned_server'] = df_servers.at[index_server, 'name']
    
    last_server = df_servers.iloc[-1]['name']
    for index_names, row_names in df_names.iterrows():
        if pd.isna(row_names['assigned_server']):
            df_names.at[index_names, 'assigned_server'] = last_server

df = pd.merge(df_names, df_servers, how='outer')
df = df.sort_values('angle')

AssignNamesToServers()

#Todo: print the final DataFrame

df

,name,type,hash,angle
105,Velma,key,1246335,4.486806
104,Velma,key,1246335,4.486806
89,Sharon,key,1364440,4.911984
90,Sharon,key,1364440,4.911984
12,Brenda,key,2749186,9.897070
...,...,...,...,...
22,Debera,key,95261585,342.941706
21,Dean,key,95932462,345.356863
84,Server_Norman,server,97261689,350.142080
3,Alicia,key,99208525,357.150690


### Task 02: Balanced Consistent Hashing
#### Extend the code from Task 01 to ensure object keys are evenly distributed among servers

In [164]:
count_names = len(df_names.index)
count_servers = len(df_servers.index)

for index_names, row_names in df_names.iterrows():
    if index_names % 10 == 0:
        server_index = (index_names // 10) % count_servers
        df_servers.at[server_index, 'hash'] = df_names.at[index_names, 'hash'] + 1

df_servers['angle'] = getAngles(df_servers['hash'])

df = pd.merge(df_names, df_servers, how='outer')
df = df.sort_values('angle')

AssignNamesToServers()
df

# Pure chaos Fuck it!!
# Completely useless task

,name,type,hash,angle,assigned_server
105,Velma,key,1246335,4.486806,Server_Shayna
104,Velma,key,1246335,4.486806,Server_Shayna
90,Sharon,key,1364440,4.911984,Server_Shayna
89,Sharon,key,1364440,4.911984,Server_Shayna
12,Brenda,key,2749186,9.897070,Server_Shayna
...,...,...,...,...,...
60,Maria,key,93139009,335.300432,Server_Shayna
5,Andrew,key,93887258,337.994129,Server_Shayna
22,Debera,key,95261585,342.941706,Server_Shayna
21,Dean,key,95932462,345.356863,Server_Shayna


### Task 03: Weighted Balanced Consistent Hashing
#### Extend the code from Task 01 to ensure object keys are evenly distributed among servers but also with a weighting of the single servers.

,name,type,hash,angle
0,Server_Helen,server,16298579,11.815081
4,Server_Robert,server,10177827,153.222520
5,Server_Richard,server,11871956,161.402317
9,Server_Eva,server,91829435,177.449609
6,Server_Dedra,server,4527453,242.846590
1,Server_Susan,server,49661912,255.511130
2,Server_Brent,server,63005364,285.438769
3,Server_Nancy,server,32199610,323.689946
8,Server_Norman,server,44134372,350.142080
7,Server_Shayna,server,55141115,358.980768
